In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from datetime import date

In [ ]:
# Import the function to get connect to the db
from snowexsql.db import get_db

# This is what you will use for all of hackweek to access the db
db_name = 'snow:hackweek@db.snowexdata.org/snowex'

In [ ]:
from snowexsql.data import PointData, ImageData

In [ ]:
from metloom.pointdata import SnotelPointData
import geopandas as gpd

In [ ]:
# import necessary libraries
from snowexsql.data import SiteData, ImageData
from snowexsql.conversions import raster_to_rasterio, query_to_geopandas
from geoalchemy2.types import Raster
import geoalchemy2.functions as gfunc
from geoalchemy2.shape import from_shape,to_shape
from rasterio.plot import show
from sqlalchemy.sql import func

In [ ]:
 def get_aso_depths(dt, snotel_code="622:CO:SNTL", crs=26912, buffer_dist=1000):
    """
    Args:
        dt: datetime or date object
        snotel_code: desired NRCS api station code
        crs: integer crs
        buffer_dist: buffer distance in same units as crs (default 1000 m)
    """
    # Pull in Snotel point 
    sntl_point = SnotelPointData(snotel_code, "dummy name")
    geom = sntl_point.metadata
    geom = gpd.GeoSeries(geom).set_crs(4326).to_crs(crs).geometry.values[0]

    # grab a session
    engine, session = get_db(db_name)

    # Building a buffer which will give us a buffer object around our point
    buffer = session.query(gfunc.ST_SetSRID(gfunc.ST_Buffer(from_shape(geom), buffer_dist), crs)).all()[0][0]

    # Convert to a shapely shapefile object
    circle = to_shape(buffer)

    # Convert to a geopandas dataframe
    df_circle = gpd.GeoSeries(circle)

    # Grab the rasters, union them and convert them as tiff when done
    q = session.query(func.ST_AsTiff(func.ST_Union(ImageData.raster, type_=Raster)))

    # Only grab rasters that are the bare earth DEM from USGS
    q = q.filter(ImageData.type == 'depth').filter(ImageData.observers=='ASO Inc.')
    q = q.filter(ImageData.date == dt)

    # And grab rasters touching the circle
    q = q.filter(gfunc.ST_Intersects(ImageData.raster, buffer))

    # Execute the query
    rasters = q.all()

    # Get the rasterio object of the raster
    dataset = raster_to_rasterio(session, rasters)[0]
    return dataset
    

# get the dataset of 
dataset = get_aso_depths(date(2020, 2, 2))

In [ ]:
import numpy as np

def rasterio_to_df(dataset):
    data = dataset.read(1)
    data[data < 0 ] = np.nan
    data_shape = data.shape
    crs = dataset.crs
    cols, rows = np.meshgrid(np.arange(data_shape[0]), np.arange(data_shape[1]))
    xs, ys = rasterio.transform.xy(dataset.transform, rows, cols)

    xs = np.array([np.array(xi) for xi in xs])
    ys = np.array([np.array(yi) for yi in ys])
    values = data.flatten()
    points = gpd.points_from_xy(xs.flatten(), ys.flatten())
    df_depths = gpd.GeoDataFrame(geometry=points)
    df_depths["depth"] = values
    df_depths = df_depths.set_crs(crs)
    df_depths = df_depths.dropna()
    return df_depths

In [ ]:
# convert the first band of the rasterio dataset to a geodataframe
df_depths = rasterio_to_df(dataset)
# convert crs
df_depths = df_depths.to_crs(4326)
df_depths.head()

In [ ]:
print(len(df_depths))
df_depths.plot(column="depth", cmap="viridis")

In [ ]:

# Create a single plot to add everything to
fig, ax = plt.subplots(figsize=(8,8))

# Plot the DEM
img = show(dataset, ax=ax, transform=dataset.transform, cmap='terrain')

# Plot the contours of the DEM (Just for kicks!) at 10m intervals
show(dataset, contour=True, levels=[s for s in np.arange(3000, 4000, 10)], colors='dimgray', ax=ax, transform=dataset.transform)

# Plot the circle as blue with slight transparency
df_circle.plot(ax=ax, color='b', alpha=0.4, edgecolor='black')

